## sequence prediction

In [1]:
from IPython.display import clear_output

import tensorflow as tf
from tensorflow import nn
import numpy as np
import math

lstm_size  = 512
n_layers   = 3
n_epochs   = 10
seq_len    = 50
batch_size = 50
lr = 0.01
grad_norm = 5

### utils

In [2]:
def xavier(n_in, n_out):
    init_range = 4*math.sqrt(6.0/(n_in + n_out))
    return tf.random_uniform([n_in, n_out], -init_range, init_range)

### data loading

In [3]:
with open('shakespeare.txt') as f:
    words_full = [c for c in list(f.read())]
    vocab = []
    for c in words_full:
        if c not in vocab:
            vocab.append(c)
    vocab_size = len(vocab)    
    
    #make batches
    tr_x = np.array([vocab.index(el) for el in words_full], dtype='int32')
    #target is just data shifted one element to the right
    tr_y = np.roll(tr_x, 1)
    sp = [i*batch_size*seq_len for i in range(1,len(words_full)//(batch_size*seq_len) + 1)]
    n_batches = len(tr_x)//(batch_size*seq_len)
    if(len(tr_x) % batch_size*seq_len != 0):
        # drop last small batch
        tr_x = np.array(np.split(tr_x, sp)[:-1]).reshape(n_batches, seq_len, batch_size)
        tr_y = np.array(np.split(tr_y, sp)[:-1]).reshape(n_batches, seq_len, batch_size)

### model

In [4]:
#create model
tf.reset_default_graph()

lstm = nn.rnn_cell.BasicLSTMCell(lstm_size)
stacked_lstm = nn.rnn_cell.MultiRNNCell([lstm] * n_layers)
input_data = tf.placeholder(tf.int32, [batch_size, seq_len])
targets = tf.placeholder(tf.int32, [batch_size, seq_len])
initial_state = state = stacked_lstm.zero_state(batch_size, tf.float32)
embedding = tf.get_variable("embedding", [vocab_size, lstm_size])
inputs = tf.nn.embedding_lookup(embedding, input_data)

outputs = []
for i in range(seq_len):
    if i > 0:
        tf.get_variable_scope().reuse_variables()
    cell_output, state = stacked_lstm(inputs[:,i,:], state)
    outputs.append(cell_output)
final_state = state

output = tf.reshape(tf.concat(1, outputs), [-1, lstm_size])
softmax_w = tf.Variable(tf.random_normal([lstm_size, vocab_size], stddev=0.5), dtype=tf.float32)
softmax_b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)
z = tf.matmul(output, softmax_w) + softmax_b
probs = tf.nn.softmax(z)
loss = tf.nn.seq2seq.sequence_loss_by_example([z], [tf.reshape(targets, [-1])], [tf.ones([batch_size * seq_len])] )
cost = tf.reduce_sum(loss) / batch_size
lr = tf.Variable(lr, trainable=False)
optimizer = tf.train.GradientDescentOptimizer(lr)
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_norm)
train_op = optimizer.apply_gradients(zip(grads, tvars))


### training

In [5]:
init = tf.initialize_all_variables()
saver = tf.train.Saver(tf.all_variables())
#train
with tf.Session() as s:
    s.run(init)
    state = initial_state.eval()
    for i in range(n_epochs):
        ep_cost = 0
        for j in range(n_batches):
            print("\r", 'Epoch %d, Batch %d' % (i+1, j+1), end='')
            _, _, co, _ = s.run([output, final_state, cost, train_op], feed_dict={input_data:tr_x[j], targets:tr_y[j], initial_state: state})
            ep_cost+=co
        print("\r", 'Epoch %d, cost per epoch %.3f' % (i+1, ep_cost/n_batches))
        saver.save(s, 'model')

 Epoch 1, Batch 12.8026e-44
 Epoch 1, Batch 22.52234e-44
 Epoch 1, Batch 37.00649e-45
 Epoch 1, Batch 42.38221e-44
 Epoch 1, Batch 52.66247e-44
 Epoch 1, Batch 67.00649e-45
 Epoch 1, Batch 71.54143e-44
 Epoch 1, Batch 81.4013e-45
 Epoch 1, Batch 91.54143e-44
 Epoch 1, Batch 107.00649e-45
 Epoch 1, Batch 111.82169e-44
 Epoch 1, Batch 123.08286e-44
 Epoch 1, Batch 135.1848e-44
 Epoch 1, Batch 145.04467e-44
 Epoch 1, Batch 153.08286e-44
 Epoch 1, Batch 161.12104e-44
 Epoch 1, Batch 171.54143e-44
 Epoch 1, Batch 187.00649e-45
 Epoch 1, Batch 197.00649e-45
 Epoch 1, Batch 203.22299e-44
 Epoch 1, Batch 211.54143e-44
 Epoch 1, Batch 221.96182e-44
 Epoch 1, Batch 236.5861e-44
 Epoch 1, Batch 241.54143e-44
 Epoch 1, Batch 254.2039e-45
 Epoch 1, Batch 261.54143e-44
 Epoch 1, Batch 271.54143e-44
 Epoch 1, Batch 284.2039e-44
 Epoch 1, Batch 291.54143e-44
 Epoch 1, Batch 309.80909e-45
 Epoch 1, Batch 311.12104e-44
 Epoch 1, Batch 327.00649e-45
 Epoch 1, Batch 331.26117e-44
 Epoch 1, Batch 344.2039e

KeyboardInterrupt: 

### testing

In [ ]:
#create model
tf.reset_default_graph()

lstm = nn.rnn_cell.BasicLSTMCell(lstm_size)
stacked_lstm = nn.rnn_cell.MultiRNNCell([lstm] * n_layers)
input_data = tf.placeholder(tf.int32, [1, 1])
initial_state = state = stacked_lstm.zero_state(1, tf.float32)

embedding = tf.get_variable("embedding", [vocab_size, lstm_size])
inputs = tf.nn.embedding_lookup(embedding, input_data)
cell_output, state = stacked_lstm(inputs[:,0,:], state)
outputs = [cell_output]
final_state = state

output = tf.reshape(tf.concat(1, outputs), [-1, lstm_size])
softmax_w = tf.Variable(tf.random_normal([lstm_size, vocab_size], stddev=0.35), dtype=tf.float32)
softmax_b = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)
z = tf.matmul(output, softmax_w) + softmax_b
probs = tf.nn.softmax(z)

In [ ]:
init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    state = initial_state.eval()
    saver = tf.train.Saver(tf.all_variables())
    ckpt = tf.train.get_checkpoint_state('model')
    saver.restore(sess, 'model')

    inpt = 'Te'
    sample_size = 50
    for c in inpt[:-1]:
        x = np.zeros((1,1), dtype='int32')
        x [0,0] = vocab.index(c)
        state = sess.run(final_state, feed_dict={input_data:x, initial_state:state})
    
    res = inpt
    char = inpt[-1]

    for i in range(sample_size):
        x = np.zeros((1,1), dtype='int32')
        x[0, 0] = vocab.index(char)
        probabilities, state = sess.run([probs, final_state], feed_dict={input_data:x, initial_state:state})
        p = probabilities[0]
        sample = np.argmax(p)
        pred = vocab[sample]
        res += pred
        char = pred
    print(res)